In [2]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings('ignore')

# Загрузка данных
train_data = pd.read_csv('/content/drive/MyDrive/задание 11/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/задание 11/test.csv')


In [11]:
pip install pandas numpy nltk scikit-learn xgboost catboost


In [12]:
pip install xgboost --upgrade


In [6]:
# Загрузка необходимых ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron

True

In [ ]:
stopwords.words('russian')


['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [26]:
pip install spacy


In [28]:
!python -m spacy download ru_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 34.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
import spacy

nlp = spacy.load("ru_core_news_md")

In [31]:

def preprocess_text(text):
    # Применение модели SpaCy к тексту
    doc = nlp(text)

    # Предобработка текста
    processed_tokens = []
    for token in doc:
        # Удаление стоп-слов и знаков препинания
        if not token.is_stop and not token.is_punct:
            # Лемматизация и приведение к нижнему регистру
            processed_tokens.append(token.lemma_.lower())

    # Объединение токенов в строку
    return ' '.join(processed_tokens)

In [7]:


# Определение стоп-слов и лемматизатора
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'[^а-яА-Я0-9\s]', '', text.lower())  # Удаление знаков препинания
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Применение предобработки
train_data['cleaned_review'] = train_data['review'].apply(preprocess_text)
test_data['cleaned_review'] = test_data['review'].apply(preprocess_text)


In [8]:
# Векторизация данных
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer.fit_transform(train_data['cleaned_review'])
y_train = train_data['sentiment']
X_test = tfidf_vectorizer.transform(test_data['cleaned_review'])


In [15]:
# # Определение моделей
# log_reg = LogisticRegression(max_iter=1000)
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# # Использование XGBoost с поддержкой GPU
# xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method='gpu_hist', gpu_id=0)

# # Использование CatBoost с поддержкой GPU
# catboost_classifier = CatBoostClassifier(silent=True, task_type='GPU')

# # Создание ансамбля
# voting_classifier = VotingClassifier(estimators=[
#     ('log_reg', log_reg),
#     ('rf', rf_classifier),
#     ('xgb', xgb_classifier),
#     ('catboost', catboost_classifier)
# ], voting='soft')

log_reg = LogisticRegression(max_iter=1000)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Создание ансамбля
voting_classifier = VotingClassifier(estimators=[
    ('log_reg', log_reg),
    ('rf', rf_classifier)
], voting='soft')

# Обучение ансамбля с кросс-валидацией и обработкой ошибок
try:
    cv_scores = cross_val_score(voting_classifier, X_train, y_train, cv=5, scoring='f1_macro', error_score='raise')
    print(f'CV F1 Score: {np.mean(cv_scores):.4f}')
except Exception as e:
    print(f"Ошибка при кросс-валидации: {e}")

# try:
#     cv_scores = cross_val_score(voting_classifier, X_train, y_train, cv=5, scoring='f1_macro', error_score='raise')
#     print(f'CV F1 Score: {np.mean(cv_scores):.4f}')
# except Exception as e:
#     print(f"Ошибка при кросс-валидации: {e}")
print(f'CV F1 Score: {np.mean(cv_scores):.4f}')


CV F1 Score: 0.9325
CV F1 Score: 0.9325


In [10]:
print(train_data.info())
print(train_data.isnull().sum())
print(train_data['sentiment'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           24000 non-null  int64 
 1   review          24000 non-null  object
 2   sentiment       24000 non-null  int64 
 3   cleaned_review  24000 non-null  object
dtypes: int64(2), object(2)
memory usage: 750.1+ KB
None
index             0
review            0
sentiment         0
cleaned_review    0
dtype: int64
sentiment
1    8000
2    8000
0    8000
Name: count, dtype: int64


In [ ]:
# Обучение модели
voting_classifier.fit(X_train, y_train)

# Предсказания
predictions = voting_classifier.predict(X_test)


In [ ]:
# Подготовка для сохранения результатов
submission = pd.DataFrame({
    'index': test_data['index'],
    'sentiment': predictions
})

# Сохранение в файл
submission.to_csv('submission.csv', index=False)


In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Загрузка модели и токенизатора
model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

# Перемещение модели на GPU, если доступно
if torch.cuda.is_available():
    model.cuda()

# Функция для обучения модели
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

from transformers import DataCollatorWithPadding

# Обновленный класс ReviewDataset
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class ReviewDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        label = self.labels[idx]

        # Токенизация отзыва
        inputs = self.tokenizer(review, return_tensors='pt', truncation=True, padding='max_length', max_length=512)

        # Убедитесь, что возвращаемый объект - это словарь, а не список
        inputs['labels'] = torch.tensor(label)  # добавляем метку в словарь
        return {key: val.squeeze(0) for key, val in inputs.items()}  # убираем лишнюю размерность


from torch.utils.data import DataLoader

# Создание экземпляра токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Создание датасета
dataset = ReviewDataset(train_data['review'].tolist(), train_data['sentiment'].tolist(), tokenizer)

# Создание DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


from transformers import AdamW

def train_model(model, dataloader):
    model.train()
    optimizer = AdamW(model.parameters(), lr=5e-5)

    for epoch in range(3):  # количество эпох
        for batch in dataloader:
            inputs = {key: val.to(model.device) for key, val in batch.items()}  # перемещение на устройство
            labels = inputs.pop('labels')  # извлечение меток из входных данных

            optimizer.zero_grad()
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()


# Обучение модели
train_model(model, dataloader)


In [33]:
def estimate_sentiment(messages: list) -> list:
    sentiment_out = []
    model.eval()  # Переключение в режим оценки
    with torch.no_grad():
        for text in messages:
            inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
            logits = model(**inputs).logits
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            sentiment_out.extend(predictions)
    return sentiment_out

# Получение предсказаний
test_reviews = test_data['review'].tolist()
predictions = estimate_sentiment(test_reviews)


In [35]:
# Сохранение предсказаний в файл
submission = pd.DataFrame({
    'index': test_data['index'],
    'sentiment': predictions
})
submission.to_csv('submission1.csv', index=False)


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer
import torch

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer
import torch

# Загрузка данны

# Предобработка данных (можно добавить дополнительную предобработку, если нужно)
train_data['review'] = train_data['review'].str.replace('[^а-яА-Я0-9\s]', '', regex=True).str.lower()
test_data['review'] = test_data['review'].str.replace('[^а-яА-Я0-9\s]', '', regex=True).str.lower()

# Разделение на признаки и целевую переменную
X = train_data['review']
y = train_data['sentiment']

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
# Инициализация токенизатора
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

# Функция для токенизации
def tokenize_data(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Токенизация данных
train_encodings = tokenize_data(X_train)
val_encodings = tokenize_data(X_val)
test_encodings = tokenize_data(test_data['review'])


In [42]:
from torch.utils.data import Dataset

class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Создание датасетов
train_dataset = ReviewsDataset(train_encodings, y_train.values)
val_dataset = ReviewsDataset(val_encodings, y_val.values)


In [43]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Инициализация модели
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=3)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Обучение модели
trainer.train()


pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.191500,0.193372
2,0.121200,0.164935
3,0.049800,0.216708


TrainOutput(global_step=3600, training_loss=0.20051867188083836, metrics={'train_runtime': 1870.4149, 'train_samples_per_second': 30.795, 'train_steps_per_second': 1.925, 'total_flos': 3788833215283200.0, 'train_loss': 0.20051867188083836, 'epoch': 3.0})

In [44]:
# Предсказание на тестовом наборе
test_dataset = ReviewsDataset(test_encodings, [0]*len(test_data))  # Псевдонабор для теста
predictions = trainer.predict(test_dataset)

# Получаем предсказанные метки
predicted_labels = predictions.predictions.argmax(axis=-1)

# Создание файла submission.csv
submission = pd.DataFrame({
    'index': test_data['index'],
    'sentiment': predicted_labels
})

submission.to_csv('submission.csv', index=False)


In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch

In [47]:
# Инициализация токенизатора
tokenizer = RobertaTokenizer.from_pretrained('sberbank-ai/ruRoberta-large')

# Функция для токенизации
def tokenize_data(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Токенизация данных
train_encodings = tokenize_data(X_train)
val_encodings = tokenize_data(X_val)
test_encodings = tokenize_data(test_data['review'])


vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

In [48]:
from torch.utils.data import Dataset

class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Создание датасетов
train_dataset = ReviewsDataset(train_encodings, y_train.values)
val_dataset = ReviewsDataset(val_encodings, y_val.values)


In [ ]:
# Инициализация модели
model = RobertaForSequenceClassification.from_pretrained('sberbank-ai/ruRoberta-large', num_labels=3)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Обучение модели
trainer.train()


pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


In [ ]:
# Предсказание на тестовом наборе
test_dataset = ReviewsDataset(test_encodings, [0]*len(test_data))  # Псевдонабор для теста
predictions = trainer.predict(test_dataset)

# Получаем предсказанные метки
predicted_labels = predictions.predictions.argmax(axis=-1)

# Создание файла submission.csv
submission = pd.DataFrame({
    'index': test_data['index'],
    'sentiment': predicted_labels
})

submission.to_csv('submission.csv', index=False)
